Individual Planning Report Group 43 - Anson Ng Student ID (34713040) 


**Data description**

In [ ]:
#load library first
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
source('cleanup.R')

There are two datasets the player dataset, and the session dataset. The player dataset is the dataset that contains information regarding each player that has logged on to play the game. This information was gathered based upon reports from players and calculations based off of the time they logged in to play. 

This player dataset includes
| Variable | Type | Description |
|----------|------|-------------|
|experience|character|Gaming skill level: Beginner, Amateur, Pro, Veteran|
|name|character|Player’s real name|
|hashedEmail|character|Unique player ID (hashed email)|
|gender|character|Player’s gender: Male, Female, Other, Prefer not to say|
|subscribe|logical|Whether player subscribed to newsletter (TRUE/FALSE)|
|played_hours|numeric|Total hours played|
|age|numeric|Player’s age (some NAs present)|

It is important to note 
- age contains missing values; we must handle NAs.
- gender contains categories beyond Male/Female, which may need filtering.
- Some variables (e.g., name, hashedEmail) are identifiers and not predictive.

The session dataset include
| Variable | Type | Description |
|----------|------|-------------|
|hashedEmail|character|Player identifier|
|start_time|datetime|Session start (date & time)|
|end_time|datetime|Session end (date & time)|
|original_start_time|numeric|Start time in UNIX milliseconds|
|original_end_time|numeric|End time in UNIX milliseconds|

It is important to note 
- Each player can have multiple sessions.
- Start and end times are recorded in both datetime and UNIX formats, which may require wrangling.
- Since played_hours already totals each player’s game time, session-level start and end times are unnecessary for predicting subscription and would add redundant complexity.

With the steps detailed in later steps, we will be able to determine the summary statistics value of  quantitative data within our tables include 
the maximum, minimum, average and median value of the quantitative values. 

**Question** 

**Broad Question:**
What player characteristics and behaviors are most predictive of subscribing to a game-related newsletter, and how do these features differ between player types?

**Specific Question:**
Which is more predictive of subscription: a player’s gender or played_hours?

**Data and Approach:**
We will compare the predictive ability of gender and played_hours for subscription. Only the players dataset is required to determine predictability. Steps include:
- Filter missing values
- Select relevant columns: played_hours, gender, subscribe
- Convert gender to categorical and ensure played_hours is numeric
- applying a linear regression model. 
    

**Exploring Data Analysis**

The summary of quantitiave variables "played_hours" and "age" 


the R commands as stated below to wrangle and summarize the data. 

In [ ]:
#load the datasets
players <- read_csv("data/players.csv")
sessions <- read_csv("data/sessions.csv")

players
sessions    

For the dataset to be clean,  ensure each variable is in its own column, each observation is in its own row, and each measured value is in a single cell. 

Player Dataset 
- does not need to be wrangled
- each observation is in its own row 
- only one value in one cell

Session Dataset
- the start_time has two values in its column
- the end_time has two values in its column
- needs to be wrangled
  

In [ ]:
wrangled_session <- sessions |>
  separate(start_time, into = c("start_date", "start_time"), sep = " ") |>
  separate(end_time, into = c("end_date", "end_time"), sep = " ")
wrangled_session

In [ ]:
#filter and select to find (summary statistic) for "played_hours" and "age"
#the original_start_time and original_end_time refers to the timestamp, we can calculate the duration of the session and caclulate the summary statistic. 
summary_played_hours <- players |> 
select(played_hours) |> 
summarize(average_played_hours = round(mean(played_hours,na.rm = TRUE ),2), minimum_played_hours = round(min(played_hours,na.rm = TRUE ),2), median_played_hours = round(median(played_hours,na.rm = TRUE ),2), max_played_hours = round(max(played_hours,na.rm = TRUE ),2)   )

summary_age <- players |> 
select(Age) |> 
summarize(average_age = round(mean(Age,  na.rm = TRUE),2), min_age = round(min(Age,na.rm = TRUE ),2), median_age = round(median(Age,na.rm = TRUE ),2), max_age = round(max(Age,na.rm = TRUE ),2)  )

sessions <- sessions |>
    mutate(original_end_time = as.numeric(as.POSIXct(end_time, format = "%d/%m/%Y %H:%M")),
           original_start_time = as.numeric(as.POSIXct(start_time, format = "%d/%m/%Y %H:%M")))

session_duration <- sessions |>
    mutate(duration = original_end_time - original_start_time) |>
    select(duration)

summary_durations <- session_duration |> 
select(duration) |>
    summarize(min_duration_ms = min(duration, na.rm = TRUE), median_duration_ms = median(duration, na.rm = TRUE), max_duration_ms = max(duration, na.rm = TRUE))

summary_played_hours 
summary_age

summary_durations

To prepare for exploratory visualizations, we will select relevant columns, remove missing or invalid values, and convert variables to the appropriate type. 

In [ ]:
# wrangling the data
wrangled_players <- players |> 
  select(subscribe, played_hours, gender) |>
filter(!is.na(played_hours), !is.na(gender), !is.na(subscribe)) |> 
  mutate(subscribe = as.factor(subscribe),gender = as.factor(gender), played_hours = as.numeric(played_hours))

wrangled_players 

**exploratory visualizations**

In [ ]:
gender_ver_subscribe <- wrangled_players |>
ggplot(aes(x=gender, fill = subscribe)) + 
geom_bar() + 
labs(x = "gender", y="number of players", fill = "subscribe", title = "Subscription by Gender" ) + 
theme_minimal()

summary_subscribe <- wrangled_players |>
  group_by(subscribe) |>
  summarise(mean_hours = mean(played_hours, na.rm = TRUE))

hours_ver_subscribe <- ggplot(summary_subscribe, aes(x = subscribe, y = mean_hours, fill = subscribe)) +
  geom_col() +
  labs(title = "Average Played Hours by Subscription Status", x = "Subscription Status",y = "Average Played Hours (hours)",fill = "Subscribed")


In [ ]:
gender_ver_subscribe
hours_ver_subscribe

Subscription by gender shows that males are overrepresented, which may bias predictions. 

Average played hours by subscription status indicates that subscribers have higher average played hours, suggesting it is a strong predictor of subscription.

**Method and Plan**

**Question** 
 which is is more predictive in determining whether one would subcribe, the played_hours or the gender

 **Method**
 Create two linear regression models for each variable played_hours and gender. With the linear regression model, we can produce a RMSE value that will demontrate the predictor with a lower value to be more predictive of subscription likelihood. Additionally the coefficients will show how changes in the predictor affect subscription probability 
 
 *We must remember to convert the subscription to binary 1/0, this will allow linear regression model to interpret subscription status numerically and calculate predictions and errors.

**Assumptions**
Assumptions that will need to be made includes 
- the relationship between each variable and subscription is linear
- there aren't too many outliers
- there are observations for all variables

**Potential limitations or witnesses**
- Binary outcome may bias toward majority class like male
- Non-linear relationships not captured thus an over or under estimation.


 **Plan** 
 
	1.	Split wrangled data into 60% training and 40% testing.
	2.	Use K-fold cross-validation on the training set to tune and validate the model.
	3.	Fit linear regression models separately for played_hours and gender.
	4.	Predict on the testing set and calculate RMSE.
	5.	Compare models: lower RMSE and larger coefficients indicate stronger predictive power.